In [1]:
import datetime as dt
import pandas as pd
import requests
from dask.distributed import LocalCluster
from dask.distributed import Client

In [6]:
api = 'http://172.16.0.170:5000/naive'

def worker(api, query):
    start = dt.datetime.now()
    resp = requests.post(api, data={'s': query})
    elapsed = dt.datetime.now() - start
    return elapsed.total_seconds() * 1000, resp.status_code

In [7]:
configs = [(1, 1), (1, 2), (1, 4), (2, 4), (4, 4), (4, 8), (4, 16), (4, 32)]
num_q = 1024

In [8]:
config_res = []

for config in configs:
    c = LocalCluster(*config)
    client = Client(c)

    qs = ['你好'] * num_q

    start = dt.datetime.now()
    results = []

    for s in qs:
        results.append(client.submit(worker, api, s, pure=False))

    results = client.gather(results)
    elapsed = (dt.datetime.now() - start).total_seconds() * 1000

    client.close()
    c.close()

    times = [r for r, _ in results]
    status = [1 if r == 200 else 0 for _, r in results]
    df = pd.DataFrame({'latency': times, 'status': status})
    res = df[['latency']].describe()
    res.loc['QPS', 'latency'] = len(qs) / elapsed * 1000
    res.loc['success', 'latency'] = df['status'].sum() / len(df)
    config_res.append(res)
    
total_df = pd.concat(config_res, axis=1)

In [9]:
total_df

,latency,latency,latency,latency,latency,latency,latency,latency
count,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000
mean,17.759516,27.578925,56.160179,112.813187,224.604695,439.311726,857.659140,1648.645370
std,4.150736,3.130661,4.798792,7.680645,18.190926,46.892609,130.789973,367.055490
min,11.182000,16.878000,35.663000,34.733000,22.256000,27.401000,26.754000,18.021000
25%,16.260000,25.713000,53.283500,108.961500,219.469750,432.455250,872.589500,1758.503750
50%,17.637000,27.585000,56.241000,113.396000,225.936000,447.665500,885.877000,1769.267500
75%,18.889750,29.455250,59.016500,117.557000,231.711250,457.645500,899.499250,1782.400000
max,123.828000,58.035000,90.190000,130.828000,259.671000,479.659000,930.547000,1815.189000
QPS,52.159213,67.732394,68.260064,68.320713,68.895123,69.735767,70.356006,70.901312
success,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
